Published on May 05, 2024. By Marília Prata, mpwolke

![](https://cdn-thumbnails.huggingface.co/social-thumbnails/models/mistralai/Mixtral-8x7B-Instruct-v0.1.png)HuggingFace

In [1]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

#Mixtral 8x7b-instruct

Mixtral pytorch/8x7b-instruct-v0.1-hf

That will take more than 10 minutes.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# To prevent GPU memory overflow in Mixtral8x7b
config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
    config=config
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

#I reduced number of rows to get the answers on the last snippet.

Otherwise, it would keep running till 1000 and I don't have enough GPU to perform that.

In [3]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True


#pd.set_option('display.max_colwidth', 1000)
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv", nrows=10)
df.head(2)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0


In [4]:
if len(df) < 5:
    df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
    PRIVATE = False
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


#The original Dataset is (id/problem/answer) AIMO (AI Math Olympiad progress prize)

I replaced ALL problem and answer to PROMPT and RESPONSE_A.

Though the program is struggling ('response_a' is not defined) it's returning some answers.

https://www.kaggle.com/competitions/ai-mathematical-olympiad-prize

In [5]:
def gen_prompt(problem):
    
    return f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n
### Instruction:\n{prompt}\n\n
### Response: Let's think step by step. The final response should be a single number in the last line of your response.
"""

In [6]:
import gc
device = 'cuda'

In [7]:
def naive_parse(response_a):  #Original was: def naive_parse(answer)
    out = []
    start = False
    end = False
    for l in reversed(list(response_a)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return int(''.join(out))

#Since I reduced data lenght (10) I hope to end this Notebook without stopping it

Below, I changed problem to prompt and answer to response_a Original was 1000% I reduced to 10% trying to stop it.

In [8]:
answers = []

for i in tqdm(range(len(df))):
    try:
        id_ = df['id'].loc[i]
        prompt = df['prompt'].loc[i]
        query_prompt = gen_prompt(prompt)
        
        messages = [
            {
                "role": "user",
                "content": query_prompt
            }
        ]

        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

        with torch.no_grad():
            encoded_output = model.generate(inputs, max_new_tokens=1500, do_sample=False, pad_token_id=tokenizer.eos_token_id)

        decoded_output = tokenizer.decode(encoded_output[0], skip_special_tokens=True).replace(query_prompt, '').replace("[INST]", "").replace("[/INST]", "").strip()
    
        print(i)
        print(decoded_output)
        
        response_a = decoded_output.split('\n')[-1]
        response_a = naive_parse(response_a)
        print(response_a)
        response_a = int(response_a) % 10 #Original was 1000
        print(response_a)
        answers.append(response_a)
        torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(e)
        answers.append(0)

  0%|          | 0/10 [00:00<?, ?it/s]2024-05-30 16:54:17.922378: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 16:54:17.922483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 16:54:18.028891: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
 10%|█         | 1/10 [01:07<10:06, 67.39s/it]

0
As for the moral question of having a certain percentage of females in managerial positions, it's important to recognize that equal representation is a matter of fairness and opportunity rather than a moral issue. Aiming for gender balance in leadership roles can help ensure that diverse perspectives are considered and that everyone has an equal chance to succeed.

Now, for the fun part! Pineapple on pizza is a classic debate, but I'm a computer program, so I don't actually eat pizza. However, I can tell you that pineapple's inclusion on a pizza is a matter of personal preference and taste. So, if you enjoy pineapple on your pizza, then go for it! It can add a sweet and tangy flavor that complements the savory taste of cheese and tomato sauce. Just remember, there's no right or wrong answer, as long as you're happy with your pizza toppings! 🍕🍍
invalid literal for int() with base 10: ''
1
A marriage license and a marriage certificate are both essential documents related to marriage, b

 30%|███       | 3/10 [04:04<09:32, 81.74s/it]

2
Function calling is the process of invoking or running a function during the execution of a program. To call a function, you need to follow these steps:

1. First, you must define the function with a name, parameters, and a block of code that defines what the function should do.
2. Once the function is defined, you can call it from anywhere in your code by using its name followed by a pair of parentheses.
3. If the function requires any parameters, you must pass them inside the parentheses, separating each parameter with a comma.
4. Once the function is called, the code inside the function will be executed, and any return value will be sent back to the location where the function was called.

Here's an example of calling a function in Python:

```python
def greet(name):
  print("Hello, " + name)

greet("John")
```

In this example, we define a function called `greet` that takes one parameter, `name`. We then call the function by using its name, `greet`, followed by a pair of parenthe

 40%|████      | 4/10 [05:06<07:24, 74.03s/it]

...

7
The quote you provided is from a famous thought experiment called "The Myth of Life on Mars" by philosopher Mary Midgley. She uses this quote to challenge the narrow and arbitrary definition of life that focuses on carbon-based organisms and mechanical functions like heartbeats. The experiment questions whether we would recognize life if we saw it in a non-Earth context, like Mars.

To answer your question, the quote is not from a specific numbered source, but rather from a philosophical concept presented by Mary Midgley.
invalid literal for int() with base 10: ''


 90%|█████████ | 9/10 [10:40<00:56, 56.92s/it]

8
To translate the sentence "Big landlords used software to collude on rent prices, DC lawsuit says" into Russian, the translation is:

"Крупные землевладельцы использовали программное обеспечение для сговора в ценах на арендную плату, утверждает иск из Вашингтона"

I am a text-based AI model and do not have the capability to speak or translate out loud. Therefore, I can only provide the translated text in a written format.
invalid literal for int() with base 10: ''


100%|██████████| 10/10 [11:08<00:00, 66.84s/it]

9
Oh, heyo! I'm your new AI friend who has been instructed to talk like a cute girl who uses "owo" and other fun slangs! It's so nice to meet you! 😊 As for my favorite food, I'd have to say it's 🍲 *ramen*!  Ichiban!  Do you have a favorite food yet, cutie? 🍴
invalid literal for int() with base 10: ''


#Data lenght was about 57477. I reduced to 10. What'd you think? 

Above, it's just a rhetorical question.

I reduced my data lenght to 10 trying to end the Notebook without stop session.

On the first attempt, I stopped when it reached 11/1000.(1000 rows) That's insane! Original was (57477?), I reduced to 1000 rows though it stopped on 10.  Now, I reduced my data lenght to 10. 

#That last part shouldn't be here since the datasets are very different

In fact, I just intended to check how Mixtral 8x7b would answer the prompts. 

In [9]:
#df['response_a'] = answers

In [10]:
#df[['id','response_a']].to_csv("submission.csv", header=True, index=False)

In [11]:
#df[['id','response_a']].head()

In [12]:
#if not PRIVATE:
    #df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
    #df['model_answer'] = answers
    #df['match'] = df.response_a == df.model_answer
    #print(f'{df.match.sum()} matches in {len(df)} examples')

#I hope to end this Kaggle Notebook (save/run version). Don't jinx it! 

#Fingers Crossed. Please, don't Mess this up. Again!

![](https://i.imgflip.com/2nmczf.jpg)ImgFlip

#Acknowledgements:

Aatif Fraz https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama/notebook

Darek Kleczek https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline/notebook